In [2]:
import sys
!{str(sys.executable)} -m pip install sentence_transformers

In [ ]:
# --- 4. SimCSE + IVF-PQ -----------------------------------
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

train = pd.read_csv("./data/train.csv")
texts = (train["제목"].fillna("") + " " + train["키워드"].fillna("")).tolist()

sim_model = SentenceTransformer("jhgan/ko-sbert-sts")
sim_emb   = sim_model.encode(texts, batch_size=256, normalize_embeddings=True)

d = sim_emb.shape[1]
quantizer = faiss.IndexFlatIP(d)
index_ivfpq = faiss.IndexIVFPQ(quantizer, d, 4096, 32, 8)
index_ivfpq.train(sim_emb)
index_ivfpq.add(sim_emb)
index_ivfpq.nprobe = 8

new_vec = sim_model.encode(["새로 들어온 아이디어"], normalize_embeddings=True)
D, I = index_ivfpq.search(new_vec, k=10)
display(I, D)

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("BM-K/KoSimCSE-roberta")  # 원하는 모델로 교체
sents = ["전통 카페를 열고 싶어요", "커피 전문점 창업 아이디어"]
embs  = model.encode(sents, normalize_embeddings=True)
sim   = float(util.cos_sim(embs[0], embs[1]))
print(f"코사인 유사도: {sim:.3f}")

No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


코사인 유사도: 0.099
